# Kaggler's Guide to LightGBM Hyperparameter Tuning with Optuna in 2021
## Maximize LGBM's performance  TODO
![](images/pixabay.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pixabay?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pixabay</a>
        on 
        <a href='https://www.pexels.com/photo/silhouette-of-person-holding-sparkler-digital-wallpaepr-266429/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>

# Setup

# Introduction

# Overview of the most important parameters

# Param1

# Param2

# Param3

# Param4

# Optuna, creating the grid

# Creating Optuna study and run trials

# Visualize the results